In [5]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from dateutil import parser
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

from utils import pdf_open
from utils import header_validations
from utils import directors_report_validation
from utils import signatory_and_date_validation
from utils import statement_order_validation
from utils import statement__tables_validation
from utils import statement_index_and_header_validation
from utils import statement_table_extraction,statement_index_and_header_validation
from utils import all_table_extraction,extract_subtables_from_main_table
from utils import cross_checking
from utils import run_validation


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\ashil.p\OneDrive - HLBHamt\projects\FS Validation\utils\Statement_table_validation.py:76: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [6]:
pdf_path = r"data\Fakhruddin test.pdf"

In [7]:
# expected_values = {
#     "Company Name": "FAKHRUDDIN PROPERTIES DEVELOPMENT L.L.C",
#     "Emirate": "Dubai - United Arab Emirates",
#     "Year End": "For the year ended December 31, 2023"
# }

file=pdf_open(pdf_path)
header_check = header_validations(pdf_path)
header_check


,Page Number,Missing Fields
0,1,"Company Name, Emirate, Year End"
1,2,"Company Name, Emirate, Year End"
2,3,"Company Name, Emirate, Year End"
3,4,Year End
4,5,Year End
5,6,Year End
6,7,Year End
7,8,Year End


In [8]:
directors_report = directors_report_validation(pdf_path)
directors_report

,Page number,Heading
0,1,DIRECTOR'S REPORT


In [9]:
# Year_end= "For the year ended December 31, 2023"

# date = parser.parse(Year_end, fuzzy=True)
# date=date.date() 


# report_header = "Independent Auditor's Report (continued)"

# signatory = {
#     'Vijay Anand': '[Reg. No. 654]',
#     'Hisham': '[Reg. No. 5397]',
#     'Jay Krishnan': '[Reg. No. 5717]',
#     'Manoj Kumar': '[Reg. No. 5713]',
#     'Reg number' :"[Firm Reg. No. LC0075-01]",
#     'Date': date
# }

signatory_status, date_status = signatory_and_date_validation(pdf_path)
display(signatory_status)

display(date_status)

,Name,Reg. NO,Name Status,Reg No Status
0,Vijay Anand,[Reg. No. 654],Present,Present
1,Hisham,[Reg. No. 5397],Not Present,Not Present
2,Jay Krishnan,[Reg. No. 5717],Not Present,Not Present
3,Manoj Kumar,[Reg. No. 5713],Not Present,Not Present


,Date Status,Firm Reg number Status
0,Date Condition True,Present


In [6]:
main_statements_order = [
    'Statement of Financial Position',
    'Statement of Comprehensive Income',
    'Statement of Changes in Equity',
    'Statement of Cash Flows',
    'Notes to the Financial Statements',
]

all_possible_headers = main_statements_order + [s + ' (continued)' for s in main_statements_order]


result = statement_order_validation(pdf_path, main_statements_order, all_possible_headers)
result

,Page_NO,Header_Found,Main_Header,Validation
0,3,None,None,Invalid
1,4,None,None,Invalid
2,5,None,None,Invalid
3,6,None,None,Invalid
4,7,None,None,Invalid
5,8,None,None,Invalid
6,9,None,None,Invalid
7,10,Statement of Financial Position,Statement of Financial Position,Valid
8,11,Statement of Comprehensive Income,Statement of Comprehensive Income,Valid
9,12,Statement of Changes in Equity,Statement of Changes in Equity,Valid


In [7]:
df = statement_order_validation(pdf_path, main_statements_order, all_possible_headers)
df = df.dropna(subset=['Header_Found'])

tables_dict = statement_table_extraction(pdf_path, df)
for table_name, table_df in tables_dict.items():
    # table_df.insert(0, 'row_number', range(1, len(table_df) + 1))
    
    print(table_name)
 
    display(table_df)

Statement of Financial Position


,index,Description,Note,Year,Previous year
0,1,Assets,,NaN,NaN
1,2,Non current assets,,NaN,NaN
2,3,"Property, plant and equipment",5,2465843.0,2160287.0
3,4,Right of use assets,6.1,6082310.0,6797875.0
4,5,Total non current assets,,8548153.0,8958162.0
5,6,Current assets,,NaN,NaN
6,7,Development properties,7,474368951.0,238877621.0
7,8,Other receivables,8,66574227.0,30961654.0
8,9,Cash and cash equivalents,9,120838302.0,39655001.0
9,10,Total current assets,,661781480.0,309494276.0


Statement of Comprehensive Income


,index,Description,Note,Year,Previous year
0,1,Revenue,14,59661189.0,NaN
1,2,Direct expenses,15,-38772052.0,NaN
2,3,Gross profit/(loss),,20889137.0,NaN
3,4,Income from penalty charges,16,3622545.0,6500.0
4,5,Commission paid,17,-20520842.0,-9513762.0
5,6,"General, selling and administrative expenses",18,-860098.0,-519176.0
6,7,Depreciation,20,-1249559.0,-556057.0
7,8,Staff cost,19,-3000466.0,-1812639.0
8,9,Profit/(loss) from operations,,-1119283.0,-12395134.0
9,10,Finance cost,21,-471111.0,-264502.0


Statement of Changes in Equity


,index,,Share capital,deficit,current account,Total
0,1,Capital introduced during the period,300000.0,NaN,NaN,300000.0
1,2,Total comprehensive income/(loss) for the period,NaN,-12659636.0,NaN,-12659636.0
2,3,Net movements in current account,NaN,NaN,9523843.0,9523843.0
3,4,"Balance as at December 31, 2022 (Restated)",300000.0,-12659636.0,9523843.0,-2835793.0
4,5,Total comprehensive income/(loss) for the year,NaN,-1590394.0,NaN,-1590394.0
5,6,Net movements in current account,NaN,NaN,55460040.0,55460040.0
6,7,"Balance as at December 31, 2023",300000.0,-14250030.0,64983883.0,51033500.0
7,8,The accompanying notes form an integral part o...,NaN,NaN,NaN,NaN


Statement of Cash Flows


,index,Description,Note,Year,Previous year
0,1,Cash flows from operating activities,,NaN,NaN
1,2,Profit/(loss) for the year/period,,-1590394.0,-12659636.0
2,3,Adjustments for:,,NaN,NaN
3,4,Depreciation,20,1249559.0,556057.0
4,5,Finance cost,21,471111.0,264502.0
5,6,Provision for employees’ end of service indemnity,11,120880.0,NaN
6,7,Operating cash flows before changes in working...,,251156.0,-11839077.0
7,8,Working capital changes:,,NaN,NaN
8,9,Other receivables,8,-35612573.0,-30961654.0
9,10,Development properties,7,-235491330.0,-238877621.0


Notes to the Financial Statements


11,index,,,Sl. No.,Shareholders,Nationality,Shares,Amount
0,1,,,,,,NaN,NaN
1,2,,,1.,Fakhruddin Holdings L.L.C.,U.A.E,270.0,270000.0
2,3,,,2.,Hamid Properties Limited,U.A.E,30.0,30000.0
3,4,,,,Total,,300.0,300000.0


In [ ]:
summary, details = validate_all_tables(tables_dict)

display(summary)
display(details)


✅ Summary Report


,Table Name,Status
0,Statement of Financial Position,Correct
1,Statement of Comprehensive Income,Correct
2,Statement of Changes in Equity,Wrong
3,Statement of Cash Flows,Wrong
4,Notes to the Financial Statements,Correct



❌ Detailed Errors (if any)


,Table Name,Row Index,Column,Expected Value,Found Value
0,Statement of Changes in Equity,7,Total,51033853.0,51033500.0
1,Statement of Cash Flows,26,Year,120838302.0,120838500.0


In [59]:
# Each_page_tables = all_table_extraction(pdf_path, extracted_table_validation)
# for page, table in Each_page_tables.items():
#     print(f"\n📄 Page Number: {page}")
#     display(table)



In [ ]:
# Each_page_tables = all_table_extraction(pdf_path, extracted_table_validation)
final_subtables = extract_subtables_from_main_table(Each_page_tables)

for name,table in final_subtables.items():
    print(name)
    display(table)

5.  Property, plant and equipment


,"5. Property, plant and equipment",1,2,3,4,5
0,"December 31, 2023",NaN,NaN,NaN,NaN,NaN
1,Cost,NaN,NaN,NaN,NaN,NaN
2,"As at December 31, 2022",2017311.0,129778.0,63848.0,147624.0,2358561.0
3,Additions,782795.0,28627.0,28128.0,NaN,839550.0
4,"As at December 31, 2023",2800106.0,158405.0,91976.0,147624.0,3198111.0
5,Accumulated depreciation,NaN,NaN,NaN,NaN,NaN
6,"As at December 31, 2022",-169587.0,-10910.0,-5367.0,-12410.0,-198274.0
7,Depreciation for the year (Note 20),-457353.0,-30263.0,-16772.0,-29606.0,-533994.0
8,"As at December 31, 2023",-626940.0,-41173.0,-22139.0,-42016.0,-732268.0
9,Carrying amount:,NaN,NaN,NaN,NaN,NaN


5.  Property, plant and equipment (continued)


,"5. Property, plant and equipment (continued)",1,2,3,4,5
0,"December 31, 2022 (Restated)",NaN,NaN,NaN,NaN,NaN
1,Cost,NaN,NaN,NaN,NaN,NaN
2,Additions,2017311.0,129778.0,63848.0,147624.0,2358561.0
3,"As at December 31, 2022",2017311.0,129778.0,63848.0,147624.0,2358561.0
4,Accumulated depreciation,NaN,NaN,NaN,NaN,NaN
5,Depreciation for the period (Note 20),-169587.0,-10910.0,-5367.0,-12410.0,-198274.0
6,"As at December 31, 2022",-169587.0,-10910.0,-5367.0,-12410.0,-198274.0
7,Carrying amount:,NaN,NaN,NaN,NaN,NaN
8,"As at December 31, 2022",1847724.0,118868.0,58481.0,135214.0,2160287.0


6.1.  Right of use asset


...

,24. Financial instruments (continued),1,2
0,Categories of financial instruments (continued),NaN,NaN
1,NaN,NaN,NaN
2,Financial assets,NaN,NaN
3,Measured at amortised cost:,NaN,NaN
4,Cash and cash equivalents,120838302.0,39655001.0
5,"Other receivables (less prepayments, contract ...",NaN,NaN
6,receivable),172863.0,107995.0
7,NaN,121011165.0,39762996.0
8,Financial liabilities,NaN,NaN
9,Accounts and other payables,NaN,NaN


25.5.  Liquidity risk (continued)


,25.5. Liquidity risk (continued),1
0,liabilities as they fall due.,NaN
1,"As at December 31, 2023",NaN
2,Measured at amortised cost:,NaN
3,Accounts and other payables (less advances),10496041.0
4,Due to related parties,225561271.0
5,Lease liabilities,601211.0
6,NaN,236658523.0
7,"As at December 31, 2022",NaN
8,Measured at amortised cost:,NaN
9,Accounts and other payables (less advances),12075304.0


27.  Prior-period errors and omissions


,27. Prior-period errors and omissions,1,2,3
0,Statement of financial position,NaN,NaN,NaN
1,"As at December 31, 2022",NaN,NaN,NaN
2,"Property, plant and equipment",1984143.0,176144.0,2160287.0
3,Right-of-use assets,NaN,6797875.0,6797875.0
4,Development properties,223133451.0,15744170.0,238877621.0
5,Other receivables,1819736.0,771652.0,2591388.0
6,Inventory,21300564.0,-21300564.0,NaN
7,Long-term payables,-238983066.0,15849615.0,-223133451.0
8,Lease liabilities - Non-Current,NaN,-6527817.0,-6527817.0
9,Accounts and other payables,-12765306.0,12275306.0,-490000.0


In [65]:
cross_check_result=cross_checking(pdf_path, tables_dict)
cross_check_result

,Header,Year,Previous Year
0,"5. Property, plant and equipment",Match,Not Match
1,"5. Property, plant and equipment (continued)",Not Match,Match
2,7. Development properties,Match,Match
3,8. Other receivables,Match,Match
4,9. Cash and cash equivalents,Match,Match
5,10. Shareholders' current account,Match,Match
6,6.2. Lease liabilities,Match,Match
7,11. Provision for employees' end of service i...,Match,NaN
8,12. Accounts and other payables,Match,Match
9,6.2. Lease liabilities,Match,Match


In [ ]:
import os
import pandas as pd
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from dotenv import load_dotenv

# ---------------------------
# 1. Load environment variables
# ---------------------------
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
base_url = "https://api.groq.com/openai/v1"

# ---------------------------
# 2. Define Output Schemas
# ---------------------------
class TableSummary(BaseModel):
    Table_Name: str = Field(..., alias="Table Name")
    Status: str

class TableErrorDetail(BaseModel):
    Table_Name: str = Field(..., alias="Table Name")
    Row_Index: int = Field(..., alias="Row Index")
    Column: str
    Expected_Value: float = Field(..., alias="Expected Value")
    Found_Value: float = Field(..., alias="Found Value")

class ValidationOutput(BaseModel):
    summary: List[TableSummary]
    details: List[TableErrorDetail]

# ---------------------------
# 3. Output Parser and Prompt
# ---------------------------
output_parser = PydanticOutputParser(pydantic_object=ValidationOutput)

prompt = PromptTemplate(
    template="""
You are a financial validation expert.

You are given a table representing a financial statement.

Table Name: {table_name}

Each table:
- Has the first column as row labels (e.g., Revenue, Expenses, Total).
- The remaining columns contain numeric values (representing financial years or periods).
- Includes calculated rows such as subtotals, totals, net changes, or closing balances.

Your tasks:
1. Identify and validate rows that perform calculations (e.g., subtotals, totals, net values).
2. For each identified calculated row, verify that the displayed value is mathematically correct based on the other rows or column value.
3. Support both summation (A + B = C) and difference (A - B = C) validation patterns.
4. Use the actual numbers provided in the table (remove commas if needed) for calculation.

If all rows are mathematically correct, return a response with:
- A summary entry showing the table name and `"Status": "Correct"` and no detailed errors.

If any row has an error, return:
- A summary entry showing the table name and `"Status": "Wrong"`
- A list of error details including row index, column name, expected(which value need to displaed instead i) vs found value(in the extact value in the pdf)

You must return your output in the following structured JSON format:
{format_instructions}

--- Table Data ---
{table_markdown}
""",
    input_variables=["table_name", "table_markdown"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)


# ---------------------------
# 4. Initialize LLM
# ---------------------------
llm = ChatOpenAI(
    openai_api_key=api_key,
    openai_api_base=base_url,
    model_name="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0
)

# ---------------------------
# 5. Create LangChain Chain
# ---------------------------
chain = (
    {
        "table_name": lambda x: x["table_name"],
        "table_markdown": lambda x: x["df"].fillna("").to_markdown(index=False)
    }
    | prompt
    | llm
    | output_parser
)

# ---------------------------
# 6. Validate All Tables
# ---------------------------
def validate_all_tables(final_subtables: Dict[str, pd.DataFrame]):
    all_summary = []
    all_details = []

    for name, df in final_subtables.items():
        try:
            result = chain.invoke({"table_name": name, "df": df})
            display
            all_summary.extend(result.summary)
            all_details.extend(result.details)
        except Exception as e:
            print(f"❌ Validation failed for '{name}': {str(e)}")
            all_summary.append(TableSummary(Table_Name=name, Status=f"Error: {str(e)}"))

    summary_df = pd.DataFrame([s.dict(by_alias=True) for s in all_summary])
    details_df = pd.DataFrame([d.dict(by_alias=True) for d in all_details]) if all_details else pd.DataFrame()

    return summary_df, details_df

# ---------------------------
# 7. Example Execution
# ---------------------------
# Ensure you have this function implemented elsewhere
final_subtables = extract_subtables_from_main_table(Each_page_tables)
summary, details = validate_all_tables(final_subtables)
display(summary)
display(details)  



In [91]:
summary_df, details_df = validate_all_tables(final_subtables)
print("\n✅ Summary Report")
display(summary_df)

print("\n❌ Detailed Errors (if any)")
display(details_df.tail(30))


✅ Summary Report


,Table Name,Status
0,"5. Property, plant and equipment",Validated
1,"5. Property, plant and equipment (continued)",Validated
2,6.1. Right of use asset,Validated
3,6.2. Lease liabilities,Validated
4,7. Development properties,Validated
5,8. Other receivables,Invalid
6,9. Cash and cash equivalents,Validated
7,10. Shareholders' current account,Validated
8,10.1. Movements,Invalid
9,11. Provision for employees' end of service i...,Valid



❌ Detailed Errors (if any)


,Table Name,Row Index,Column,Expected Value,Found Value
45,12. Accounts and other payables,7,1,162697030.0,162697030.0
46,12. Accounts and other payables,7,2,84758681.0,84758681.0
47,13.2. Due to related parties,8,1,513678605.0,513678605.0
48,13.2. Due to related parties,8,2,238683065.0,238683065.0
49,13.2. Due to related parties,11,1,225561271.0,225561271.0
50,13.2. Due to related parties,11,2,6025771.0,6025771.0
51,17. Commission paid,2,2,10471884.0,9513762.0
52,17. Commission paid,2,1,20520800.0,20520800.0
53,"18. General, selling and administrative expenses",7,2,519176.0,519176.0
54,19. Staff cost,5,1,3028840.0,3000470.0


In [ ]:
import os
import pandas as pd
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from dotenv import load_dotenv

# ---------------------------
# 1. Load environment variables
# ---------------------------
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
base_url = "https://api.groq.com/openai/v1"

# ---------------------------
# 2. Define Output Schemas
# ---------------------------
class TableSummary(BaseModel):
    Table_Name: str = Field(..., alias="Table Name")
    Status: str

class TableErrorDetail(BaseModel):
    Table_Name: str = Field(..., alias="Table Name")
    Row_Index: int = Field(..., alias="Row Index")
    Column: str
    Expected_Value: float = Field(..., alias="Expected Value")
    Found_Value: float = Field(..., alias="Found Value")

class ValidationOutput(BaseModel):
    summary: List[TableSummary]
    details: List[TableErrorDetail]

# ---------------------------
# 3. Output Parser and Prompt
# ---------------------------
output_parser = PydanticOutputParser(pydantic_object=ValidationOutput)

prompt = PromptTemplate(
    template="""
You are a financial validation expert.

You are given a table representing a financial statement.

Table Name: {table_name}

Each table:
- Has the first column as row labels (e.g., Revenue, Expenses, Total).
- The remaining columns contain numeric values (representing financial years or periods).
- Includes calculated rows such as subtotals, totals, net changes, or closing balances.

Your tasks:
1. Identify and validate rows that perform calculations (e.g., subtotals, totals, net values).
2. For each identified calculated row, verify that the displayed value is mathematically correct based on the other rows or column value.
3. Support both summation (A + B = C) and difference (A - B = C) validation patterns.
4. Use the actual numbers provided in the table (remove commas if needed) for calculation.

If all rows are mathematically correct, return a response with:
- A summary entry showing the table name and `"Status": "Correct"` and no detailed errors.

If any row has an error, return:
- A summary entry showing the table name and `"Status": "Wrong"`
- A list of error details including row index, column name, expected (which value should be displayed instead) vs found value (the exact value in the table).

You must return your output in the following structured JSON format:
{format_instructions}

--- Table Data ---
{table_markdown}
""",
    input_variables=["table_name", "table_markdown"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# ---------------------------
# 4. Initialize LLM
# ---------------------------
llm = ChatOpenAI(
    openai_api_key=api_key,
    openai_api_base=base_url,
    model_name="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0
)

# ---------------------------
# 5. Create LangChain Chain
# ---------------------------
chain = (
    {
        "table_name": lambda x: x["table_name"],
        "table_markdown": lambda x: x["df"].fillna("").to_markdown(index=False)
    }
    | prompt
    | llm
    | output_parser
)

# ---------------------------
# 6. Validate All Tables
# ---------------------------
def validate_all_tables(final_subtables: Dict[str, pd.DataFrame]):
    all_summary = []
    all_details = []

    for name, df in final_subtables.items():
        try:
            result = chain.invoke({"table_name": name, "df": df})
            all_summary.extend(result.summary)
            all_details.extend(result.details)
        except Exception as e:
            print(f"❌ Validation failed for '{name}': {str(e)}")
            all_summary.append(TableSummary(Table_Name=name, Status=f"Error: {str(e)}"))

    summary_df = pd.DataFrame([s.dict(by_alias=True) for s in all_summary])
    details_df = pd.DataFrame([d.dict(by_alias=True) for d in all_details]) if all_details else pd.DataFrame()

    return summary_df, details_df

# ---------------------------
# 7. Example Execution
# ---------------------------
# Ensure extract_subtables_from_main_table is implemented elsewhere
# and Each_page_tables is a dict or list of tables from the PDF

# Example usage
# final_subtables = extract_subtables_from_main_table(Each_page_tables)
summary, details = validate_all_tables(final_subtables)
print(summary)
print(details)


❌ Validation failed for '10.  Shareholders' current account': Error code: 429 - {'error': {'message': 'Rate limit reached for model `meta-llama/llama-4-maverick-17b-128e-instruct` in organization `org_01k0gqsaw8ffv8xk0gb16rzyhc` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5687, Requested 838. Please try again in 5.247s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


ValidationError: 1 validation error for TableSummary
Table Name
  Field required [type=missing, input_value={'Table_Name': "10.  Shar...rate_limit_exceeded'}}"}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [45]:
display(details)

NameError: name 'details' is not defined